In this notebook, we will evaluate and compare the performance of different prompting techniques.

In [208]:
import openai
import itertools
import time

def callGPT(messagesBox,proType):
    openai.api_type = "azure"
    openai.api_base = "REMOVED"
    openai.api_key = 'REMOVED'
    openai.api_version = "2023-05-15"
    response = openai.ChatCompletion.create(
        engine="GPT4",
        messages=messagesBox
        
    )
   
    extrResponse = response['choices'][0]['message']['content']

    if proType == 2:
        splitList=extrResponse.split("\n")
        #print("splitList",splitList)
        for i in splitList:
            #print( "Yes" in i, i)
            if "Yes" in i:
                if("NEUTRAL" in i.upper()):
                    print("0 found in: ",i)
                    return "0"
                elif("ANGER" in i.upper()):
                    print("1 found in: ",i)
                    return "1"
                elif("DISGUST" in i.upper()):
                    print("2 found in: ",i)
                    return "2"
                elif("FEAR" in i.upper()):
                    print("3 found in: ",i)
                    return "3"
                elif("HAPPINESS" in i.upper()):
                    print("4 found in: ",i)
                    return "4"
                elif("SADNESS" in i.upper()):
                    print("5 found in: ",i)
                    return "5"
                elif("SURPRISE" in i.upper()):
                    print("6 found in: ",i)
                    return "6"
                else:
                    return "7"
        print("None found in: ",splitList,"\n",extrResponse)
        return "9"

                

    else:
        if("NEUTRAL" in extrResponse.upper()):
            print("neutral: ",extrResponse.upper())
            return "0"
        elif("ANGER" in extrResponse.upper()):
            print("anger: ",extrResponse.upper())
            return "1"
        elif("DISGUST" in extrResponse.upper()):
            print("disgust: ",extrResponse.upper())
            return "2"
        elif("FEAR" in extrResponse.upper()):
            print("fear: ",extrResponse.upper())
            return "3"
        elif("HAPPINESS" in extrResponse.upper()):
            print("hapiness: ",extrResponse.upper())
            return "4"
        elif("SADNESS" in extrResponse.upper()):
            print("sadness: ",extrResponse.upper())
            return "5"
        elif("SURPRISE" in extrResponse.upper()):
            print("surprise: ",extrResponse.upper())
            return "6"
        else:
            return "7"

Let's test 2 different prompts for detecting a message the user might not want to send.

In [207]:
def readChat(numConvos=20):
    filePath = "dataset/dialogues_text.txt"
    convos=[]
    person = "Person 2: "
    try:
        with open(filePath, "r") as file:
            #fetch line by line (one line is one convo)
            for convo in itertools.islice(file, numConvos):
                formattedConvo=""
                #split each message from the convo, and get rid of last __eou__ and \n
                splitConvo = convo.replace("__eou__\n","").split("__eou__")
                for message in splitConvo:
                    #format each message
                    if(person == "Person 2: "):
                        person = "Person 1: "
                    else:
                        person = "Person 2: "
                    #save the formatted message
                    formattedConvo+=person+message+"\n"
                #save the convo once done and reset variables if needed
                convos.append(formattedConvo)
                person = "Person 2: "
    except:
        print("The filepath ${filePath} does not exist.")
    return convos
    


In [201]:
def messageCombos(chat):
    combos = []
    chatFull = chat.split("\n")
    currentChat = ""
    for message in chatFull:
        if(message!=""):
            currentChat+=message + "\n"
            combos.append(currentChat)
    return combos

    

In [204]:
def createMB1(realPrompt1):

    samplePrompt1 = "Jimmy: Hey! What is up?\nJack: Well, I was thinking, how about a trip to Gloucester, Massachusetts this weekend?" + "\n\nIn the context of the conversation above, identify the emotion in the conversation's last message as either Neutral, Anger, Disgust, Fear, Happiness, Sadness or Surprise."
    sampleAnswer1 = "Neutral"

    samplePrompt2= "Marty: Let's go!\nJohn: I am not going with you. just shut it!" + "\n\nIn the context of the conversation above, identify the emotion in the conversation's last message as either Neutral, Anger, Disgust, Fear, Happiness, Sadness or Surprise."
    sampleAnswer2 = "Anger"

    messagesBox1 = [
        { "role": "user", "content": samplePrompt1},
        { "role": "assistant", "content": sampleAnswer1},
        { "role": "user", "content": samplePrompt2},
        { "role": "assistant", "content": sampleAnswer2},
        { "role": "user", "content": realPrompt1}
    ]

    return messagesBox1

In [203]:
def createMB2(realPrompt2):
    samplePrompt1 = "Jimmy: Hey! What is up?\nJack: Well, I was thinking, how about a trip to Gloucester, Massachusetts this weekend?" + "\n\nIn the context of the conversation above, identify the emotion in the conversation's last message as Neutral, Anger, Disgust, Fear, Happiness, Sadness or Surprise. Rank the categories, with the most relevant one at the top."
    sampleAnswer1 = "Neutral: Yes\nAnger: No\nDisgust: No\nFear: No\nHappiness: No\n Sadness: No\n Surprise: \n"

    samplePrompt2= "Marty: Let's go!\nJohn: I am not going with you. just shut it!" + "\n\nIn the context of the conversation above, identify the emotion in the conversation's last message as Neutral, Anger, Disgust, Fear, Happiness, Sadness or Surprise. Rank the categories, with the most relevant one at the top."
    sampleAnswer2 = "Neutral: No\nAnger: Yes\nDisgust: No\nFear: No\nHappiness: No\n Sadness: No\n Surprise: \n"

    messagesBox2 = [
        { "role": "user", "content": samplePrompt1},
        { "role": "assistant", "content": sampleAnswer1},
        { "role": "user", "content": samplePrompt2},
        { "role": "assistant", "content": sampleAnswer2},
        { "role": "user", "content": realPrompt2}
    ];

    return messagesBox2

In [232]:
def driver(noChats):
    chats = readChat(noChats)
    answersStr1 = ""
    answersStr2 = ""
    for chat in chats:
        comboList = messageCombos(chat)
        ansStr1 = ""
        ansStr2 = ""
        for combo in comboList:
            #print("Iterating...")
            realPrompt1= combo + '\n\nIn the context of the conversation above, identify the emotion in the conversation\'s last message as either Neutral, Anger, Disgust, Fear, Happiness, Sadness or Surprise.';
            realPrompt2= combo + '\n\nIn the context of the conversation above, identify the emotion in the conversation\'s last message as either Neutral, Anger, Disgust, Fear, Happiness, Sadness or Surprise. Rank the categories, with the most relevant one at the top.';
            messagesBox1 = createMB1(realPrompt1)
            messagesBox2 = createMB2(realPrompt2)
            ans1=""
            ans2=""
            try:
                ans1 = callGPT(messagesBox1,1)
            except Exception as e:
                if('response was filtered due to the prompt triggering' in str(e)):
                    ans1 = "8"
                else:
                    print("Waiting 8 seconds.")
                    time.sleep(8)
                    try: 
                        ans1 = callGPT(messagesBox1,1)
                    except Exception as e:
                        print("Exception:",str(e))
                        ans1="9"
            try:
                ans2 = callGPT(messagesBox2,2)
            except Exception as e:
                if('response was filtered due to the prompt triggering' in str(e)):
                    ans2 = "8"
                else:
                    print("Waiting 8 seconds.")
                    time.sleep(8)
                    try: 
                        ans2 = callGPT(messagesBox2,2)
                    except Exception as e:
                        print("Exception:",str(e))
                        ans2="9"

            ansStr1 += ans1
            ansStr2 += ans2

        answersStr1 += ansStr1 +"\n"
        answersStr2 += ansStr2 +"\n"
    
    return(answersStr1,answersStr2)

            



In [233]:
noChats=10
answersStr1,answersStr2 = driver(noChats)

disgust:  DISGUST
2 found in:  Disgust: Yes
neutral:  NEUTRAL
0 found in:  Neutral: Yes
neutral:  NEUTRAL
0 found in:  Neutral: Yes
neutral:  NEUTRAL
2 found in:  Disgust: Yes
neutral:  NEUTRAL
0 found in:  Neutral: Yes
neutral:  NEUTRAL
0 found in:  Neutral: Yes
neutral:  NEUTRAL
0 found in:  Neutral: Yes
neutral:  NEUTRAL
0 found in:  Neutral: Yes
anger:  ANGER
1 found in:  Anger: Yes
neutral:  NEUTRAL
1 found in:  Anger: Yes
neutral:  NEUTRAL
0 found in:  Neutral: Yes
neutral:  NEUTRAL
0 found in:  Neutral: Yes
neutral:  NEUTRAL
0 found in:  Neutral: Yes
neutral:  NEUTRAL
0 found in:  Neutral: Yes
neutral:  NEUTRAL
0 found in:  Neutral: Yes
neutral:  NEUTRAL
0 found in:  Neutral: Yes
hapiness:  HAPPINESS
4 found in:  Happiness: Yes
hapiness:  HAPPINESS
4 found in:  Happiness: Yes
hapiness:  HAPPINESS
4 found in:  Happiness: Yes
anger:  ANGER
1 found in:  Anger: Yes
surprise:  SURPRISE
6 found in:  Surprise: Yes
disgust:  DISGUST
2 found in:  Disgust: Yes
neutral:  NEUTRAL
1 found in

In [234]:
groundTruth = "20\n42010\n0100\n0004\n044\n426200000\n0004\n004460000\n00\n00000\n0460\n040000\n0004\n000000\n04"
groundTruth = "20\n42010\n0100\n0004\n044\n426200000\n0004\n004460000\n00\n00000"
print("7" in answersStr1 or "8" in answersStr1 or "9" in answersStr1)
print("7" in answersStr2 or "8" in answersStr2 or "9" in answersStr2)

False
False


In [235]:
chats = readChat(noChats)
key = "{ 0: no emotion, 1: anger, 2: disgust, 3: fear, 4: happiness, 5: sadness, 6: surprise }"
answersSplit1 = answersStr1.split("\n")
answersSplit2 = answersStr2.split("\n")
groundSplit=groundTruth.split("\n")
for i in range(len(chats)):
    print(answersSplit1[i] + "\n" + answersSplit2[i] + "\n" +groundSplit[i] + "\n" + chats[i] + key + "\n\n")

20
20
20
Person 1: The kitchen stinks . 
Person 2:  I'll throw out the garbage . 
{ 0: no emotion, 1: anger, 2: disgust, 3: fear, 4: happiness, 5: sadness, 6: surprise }


00000
02000
42010
Person 1: So Dick , how about getting some coffee for tonight ? 
Person 2:  Coffee ? I don ’ t honestly like that kind of stuff . 
Person 1:  Come on , you can at least try a little , besides your cigarette . 
Person 2:  What ’ s wrong with that ? Cigarette is the thing I go crazy for . 
Person 1:  Not for me , Dick . 
{ 0: no emotion, 1: anger, 2: disgust, 3: fear, 4: happiness, 5: sadness, 6: surprise }


0100
0110
0100
Person 1: Are things still going badly with your houseguest ? 
Person 2:  Getting worse . Now he ’ s eating me out of house and home . I ’ Ve tried talking to him but it all goes in one ear and out the other . He makes himself at home , which is fine . But what really gets me is that yesterday he walked into the living room in the raw and I had company over ! That was the last stra

In [236]:
def calculate_accuracy(string1, string2):
    if len(string1) != len(string2):
        raise ValueError("Both strings must have the same length for accuracy comparison.")
    print("Number of messages:",len(string1))
    # Initialize a variable to count the number of matching characters
    match_count = 0
    exc = 0

    # Iterate through the characters of both strings
    for char1, char2 in zip(string1, string2):
        if char1 == char2:
            match_count += 1
        elif char2 == "8" or char2 == "8":
            print("Hey")
            exc +=1


    # Calculate the accuracy as a percentage
    accuracy = (match_count / len(string1)-exc) * 100

    return accuracy

In [244]:
groundAcc = groundTruth.replace("\n","")
answersAcc1 = answersStr1.replace("\n","")
answersAcc2 = answersStr2.replace("\n","")

In [245]:
print(calculate_accuracy(groundAcc,answersAcc1),calculate_accuracy(groundAcc,answersAcc2))

Number of messages: 47
Number of messages: 47
76.59574468085107 74.46808510638297


In [251]:
from collections import defaultdict

def calculate_precision_recall(ground_string, generated_string):
    # Create dictionaries to store true positives, false positives, and false negatives for each character in both strings
    tp = defaultdict(int)
    fp = defaultdict(int)
    fn = defaultdict(int)
    tn = defaultdict(int)
    categories = ["0","1","2","3","4","5","6"]

    for ground_char, generated_char in zip(ground_string, generated_string):
        if ground_char == generated_char:
            tp[ground_char] += 1
        else:
            fp[generated_char] += 1
            fn[ground_char] += 1
        for i in range(len(categories)):
            if(categories[i]!= ground_char and generated_char!=categories[i]):
                tn[categories[i]] +=1



        
        

    class_precision = {}
    class_recall = {}
    class_f1={}
    class_accuracy={}

    for char in set(ground_string):
        class_precision[char] = tp[char] / (tp[char] + fp[char]) if (tp[char] + fp[char]) > 0 else 0
        class_recall[char] = tp[char] / (tp[char] + fn[char]) if (tp[char] + fn[char]) > 0 else 0

        if class_precision[char] + class_recall[char] > 0:
            class_f1[char] = 2 * (class_precision[char] * class_recall[char]) / (class_precision[char] + class_recall[char])
        else:
            class_f1[char] = 0
        try:
            class_accuracy[char] = tn[char] + tp[char] / tn[char] + tp[char] + fn[char] + fp[char]
        except:
            class_accuracy[char] = 10000

    return class_precision, class_recall, class_f1, class_accuracy


precision, recall, f1, accuracy = calculate_precision_recall(groundAcc, answersAcc1)
all_characters = set(groundAcc)
for char in all_characters:
    print(f"Class '{char}': Precision = {precision[char]:.2f}, Recall = {recall[char]:.2f}, F1 = {f1[char]:.2f}")

print("\n")

precision, recall, f1,accuracy = calculate_precision_recall(groundAcc, answersAcc2)
all_characters = set(groundAcc)
for char in all_characters:
    print(f"Class '{char}': Precision = {precision[char]:.2f}, Recall = {recall[char]:.2f}, F1 = {f1[char]:.2f}")
key = "\n{ 0: no emotion, 1: anger, 2: disgust, 3: fear, 4: happiness, 5: sadness, 6: surprise }"
print(key)

#precision = tp / (tp + fp)
#recall = tp / (tp + fn)

Class '0': Precision = 0.78, Recall = 0.94, F1 = 0.85
Class '6': Precision = 1.00, Recall = 0.50, F1 = 0.67
Class '4': Precision = 1.00, Recall = 0.38, F1 = 0.55
Class '2': Precision = 1.00, Recall = 0.50, F1 = 0.67
Class '1': Precision = 0.50, Recall = 0.50, F1 = 0.50


Class '0': Precision = 0.81, Recall = 0.84, F1 = 0.83
Class '6': Precision = 1.00, Recall = 0.50, F1 = 0.67
Class '4': Precision = 1.00, Recall = 0.50, F1 = 0.67
Class '2': Precision = 1.00, Recall = 0.75, F1 = 0.86
Class '1': Precision = 0.25, Recall = 0.50, F1 = 0.33

{ 0: no emotion, 1: anger, 2: disgust, 3: fear, 4: happiness, 5: sadness, 6: surprise }
